In [52]:
# import libraries
from rdflib import Graph,Literal,RDF,URIRef
from rdflib.namespace import FOAF,XSD,RDFS
from rdflib import Namespace
import pandas as pd

In [53]:
g = Graph()

In [54]:
KG_SDM = Namespace("http://kg_sdm.org/")
g.bind("kg_sdm",KG_SDM) # the uri will be repliced

In [55]:
# Author subclass Person
# Author: school
# Person: name
# Create a table of names and schools which will be later sliced to Author, Editor etc.

In [56]:
people_names = pd.read_csv('data_kg_sdm/authors.csv')
people_names = people_names.drop(columns='ID')
people_names.rename(columns = {'name':'person_name'}, inplace=True)

In [57]:
schools = pd.read_csv('data_kg_sdm/schools.csv')
schools = schools.drop(columns='ID')
schools.rename(columns = {'name':'school_name'}, inplace=True)

In [58]:
academics = people_names.loc[:schools.size-1]
academics['school_name'] = schools

/tmp/ipykernel_79252/2655002278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  academics['school_name'] = schools


In [59]:
academics

,person_name,school_name
0,Gautam A.,"Computer Science Department, Brigham Young Uni..."
1,Crandall J.W.,"INESC-TEC Technology and Science, Campus da FE..."
2,Goodrich M.A.,"Department of Engineering, University of Trás..."
3,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y..."
4,Hedengren J.D.,"Task Committee Secretary and Lead Engineer, Op..."
...,...,...
1456,Hedges D.M.,United States Department of Agriculture-Agric...
1457,Steffensen S.C.,United States Department of Agriculture-Agric...
1458,Puleo C.,"Agriculture and Agri-Food Canada, Lethbridge,..."
1459,Galligan C.,"Department of Crop and Soil Sciences, Washing..."


In [60]:
# We suppose 60% are Authors, 20% Reviewsrs, 10% Chair or Editor

In [61]:
total = len(academics)
nb_auth = int(total * 0.6)
nb_rev = int(total * 0.2)
nb_chair_editor = int(total * 0.1)

In [62]:
authors = academics.loc[:nb_auth]
reviewers = academics.loc[nb_auth : nb_auth + nb_rev]
chair = academics.loc[nb_auth + nb_rev : nb_auth + nb_rev + nb_chair_editor ]
editor = academics.loc[ nb_auth + nb_rev + nb_chair_editor :nb_auth + nb_rev + nb_chair_editor + nb_chair_editor]

In [63]:
editor.head()

,person_name,school_name
1314,Choudhury D.,"Department of Architecture, Building and Plan..."
1315,Chanda A.,"ISISE, University of Minho, Department of Civ..."
1316,Ruchti T.,"Utah State University, ENGR 419J, Logan, UT 8..."
1317,Upchurch W.,"Naval Undersea Warfare Center, Newport, RI, U..."
1318,Lee J.H.,"Pennsylvania State University, State College,..."


In [64]:
FOAF.Person

rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person')

In [65]:
# people = URIRef("http://example.org/editor/Bob") #should it be /people/academic/editor?
# linda = BNode()  # a GUID is generated

# name = Literal("Bob")
# age = Literal(24)

# g.add((bob, RDF.type, FOAF.Person))
# g.add((bob, FOAF.name, name))
# g.add((bob, FOAF.age, age))
# g.add((bob, FOAF.knows, linda))
# g.add((linda, RDF.type, FOAF.Person))
# g.add((linda, FOAF.name, Literal("Linda")))

# print(g.serialize())

In [66]:
url = "fadsa dadasd - "


In [67]:
def URLparse(url:str):
    return url.replace(".", "_").replace(" ", "_")

In [68]:
# editor = URIRef("http://kg_sdm.org/Editor")
academic = URIRef("http://kg_sdm.org/Academic")
# person = URIRef("http://foaf.org/FOAF.")
#     academic_node = URIRef(f"http://kg_sdm.org/Academic/{name}")
#     person_node = URIRef(f"http://kg_sdm.org/Person/{name}")

for name, school in zip(editor['person_name'], editor['school_name']):
    
#     editor_node = URIRef(f"http://kg_sdm.org/{URLparse(name)}")
    person_node = URIRef(f"http://kg_sdm.org/Editor/{URLparse(name)}")
    
    name_lit = Literal(str(name))
    school_lit = Literal(str(school))

#     g.add((editor_node, RDF.type, FOAF.Person))
#     g.add((editor_node, FOAF.name, name_lit))

#     editor_node = URIRef(f"http://kg_sdm.org/Editor/{}")
    
#     g.add((person_node, RDF.type, FOAF.Person))
    g.add((person_node, FOAF.name, name_lit))
    g.add((person_node, RDF.type, KG_SDM.Editor))
    g.add((person_node, KG_SDM.school, school_lit))
        
#     g.add((editor_node, RDF.type, KG_SDM.Editor))
#     g.add((editor_node, KG_SDM.school, school_lit))
#     g.add((editor_node, FOAF.name, name_lit))

In [69]:
print(g.serialize('abox.ttl', format="ttl"))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [70]:
FOAF.name

rdflib.term.URIRef('http://xmlns.com/foaf/0.1/name')